<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6ce3d549884b724c5c46245b83d82c4a0db835abff0e6034076182c08d9393a6
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 09:13:48
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 10.46 K (0.07%)
Current PnL: -21.18 L (-14.06%)
CY Booked + Current PnL: -9.74 L (-6.47%)
-------------------
Total profit:  1.31 L
Total loss:  -22.49 L
-------------------
Total Booked + Current PnL: 17.24 L (13.94%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.49 L (63.03%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.82,113685.0,13181.0,15927.0,0.00,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
77,TTKPRESTIG,770.00,95.52,42.0,M-SC,1.69,84207.0,-16570.0,16665.0,0.90,-16.44,19.79,0.09,245.0,-0.99,0.60,10.63,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.71,45.0,H-MC,2.93,187780.0,23110.0,20224.0,0.00,14.03,10.77,26.31,89.0,1.14,1.34,23.14,X40N,NTT,AC
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.79,56.0,H-MC,1.76,141547.0,5414.0,25705.0,0.48,3.98,18.16,22.86,91.0,0.21,1.01,17.09,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-303.99,39.0,H-SC,3.73,113560.0,-50045.0,171362.0,2.17,-30.59,150.90,74.15,139.0,-0.29,0.81,45.57,XR,NTT,CHEMICALS
33,HONAUT,58357.33,-22.51,45.0,H-SC,2.33,109200.0,-18738.0,65869.0,1.20,-14.65,60.32,36.84,143.0,-0.28,0.78,11.79,X40N,ATH,ELECTRICAL
17,CAMS,5211.76,-4.94,49.0,H-SC,1.95,108024.0,6020.0,37906.0,0.91,5.90,35.09,43.06,122.0,0.16,0.77,24.03,X40N,ATH,MISC
77,TTKPRESTIG,770.00,95.52,42.0,M-SC,1.69,84207.0,-16570.0,16665.0,0.90,-16.44,19.79,0.09,245.0,-0.99,0.60,10.63,OX40N,NTT,DURABLES
46,KPIGREEN,731.05,-4.80,35.0,H-SC,7.51,113324.0,-11973.0,70895.0,0.89,-9.56,62.56,47.03,141.0,-0.17,0.81,41.89,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.0,221.06,54.0,M-SC,2.15,143228.0,-39850.0,118822.0,-1.28,-21.77,82.96,43.14,230.0,-0.34,1.02,34.73,XY24,NTT,MISC
47,LAOPALA,464.0,106.44,40.0,H-SC,1.42,94300.0,-36805.0,88048.0,-0.99,-28.07,93.37,39.09,142.0,-0.42,0.67,21.45,AR,NTT,CERAMICS
61,SAMMAANCAP,326.0,-187.50,79.0,M-SC,9.60,151200.0,1980.0,142204.0,-0.93,1.33,94.05,96.62,208.0,0.01,1.08,63.63,XY25,NTT,FINANCE
24,DEN,75.0,91.71,37.0,M-SC,2.10,54080.0,-30384.0,65918.0,-0.59,-35.97,121.89,42.07,237.0,-0.46,0.39,14.85,AR,NTT,ENTERTAINMENT
51,NESTLEIND,1377.0,-11.79,43.0,H-LC,3.76,271440.0,6014.0,50786.0,-0.55,2.27,18.71,21.40,11.0,0.12,1.94,9.52,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.68,45.0,H-MC,2.2,134809.0,-979.0,43732.0,-0.02,-0.72,32.44,31.48,107.0,-0.02,0.96,11.31,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,65.0,H-SC,12.10,130221.0,-12258.0,31774.0,0.00,-8.60,24.40,13.70,163.0,-0.39,0.93,53.70,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,95.52,42.0,M-SC,1.69,84207.0,-16570.0,16665.0,0.90,-16.44,19.79,0.09,245.0,-0.99,0.60,10.63,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,55.0,H-SC,1.81,223650.0,-46017.0,82348.0,-0.29,-17.06,36.82,13.47,138.0,-0.56,1.60,13.12,XY24,NTT,PAINTS
66,SIS,528.0,2111.40,47.0,H-SC,3.73,89250.0,-21782.0,45393.0,0.26,-19.62,50.86,21.26,156.0,-0.48,0.64,20.59,OX40N,NTT,MISC
18,CERA,9475.0,-22.91,35.0,H-SC,1.50,140438.0,-35465.0,77494.0,0.00,-20.16,55.18,23.89,149.0,-0.46,1.00,21.29,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1254.07,70.0,M-SC,18.31,184095.0,2952.0,42508.0,0.57,1.63,23.09,25.10,235.0,0.07,1.32,47.53,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,11.89,232522.0,7560.0,160091.0,0.08,3.36,68.85,74.53,192.0,0.05,1.66,36.70,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,65.0,M-LC,4.38,210105.0,5029.0,107532.0,0.34,2.45,51.18,54.89,52.0,0.05,1.50,27.14,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.71,45.0,H-MC,2.93,187780.0,23110.0,20224.0,0.00,14.03,10.77,26.31,89.0,1.14,1.34,23.14,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,43.0,H-MC,2.19,202500.0,10758.0,26993.0,-0.26,5.61,13.33,19.69,99.0,0.40,1.45,12.79,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.82,113685.0,13181.0,15927.0,0.00,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,10.03,191712.0,11904.0,100515.0,0.02,6.62,52.43,62.52,88.0,0.12,1.37,42.96,XR,NTT,BANKS
32,HINDZINC,730.22,29.16,65.0,M-LC,4.38,210105.0,5029.0,107532.0,0.34,2.45,51.18,54.89,52.0,0.05,1.50,27.14,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.45,37.0,H-SC,9.40,126967.0,3631.0,130128.0,-0.14,2.94,102.49,108.45,119.0,0.03,0.91,25.95,AR,ATH,MISC
86,WIPRO,420.00,-15.15,41.0,M-LC,5.85,149408.0,-1537.0,110995.0,-0.04,-1.02,74.29,72.51,53.0,-0.01,1.07,4.94,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.29,30.0,H-LC,5.80,206114.0,-45654.0,98873.0,0.27,-18.13,47.97,21.13,27.0,-0.46,1.47,10.87,X40,ATH,PAINTS
83,VBL,671.64,-21.55,32.0,H-LC,9.54,282602.0,-33240.0,145907.0,-0.00,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
74,TCS,4476.75,-30.93,34.0,H-LC,12.46,274856.0,-71100.0,145949.0,0.34,-20.55,53.10,21.64,1.0,-0.49,1.97,1.23,X40,ATH,IT
30,HCLTECH,1943.91,-4.09,37.0,H-LC,10.28,219250.0,-22674.0,85946.0,0.50,-9.37,39.20,26.15,8.0,-0.26,1.57,3.61,X40,ATH,IT
44,JIOFIN,387.00,-5.04,38.0,H-LC,12.82,258043.0,-9322.0,74781.0,0.00,-3.49,28.98,24.48,48.0,-0.12,1.85,49.57,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-2.83,44.0,H-LC,1.74,156075.0,-30020.0,77507.0,0.00,-16.13,49.66,25.51,15.0,-0.39,1.12,15.02,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-6.94,43.0,H-LC,1.80,230535.0,-15616.0,94819.0,0.00,-6.34,41.13,32.18,16.0,-0.16,1.65,27.41,X200,ATH,IT
42,ITC,452.0,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
57,RELIANCE,1533.0,-15.17,39.0,H-LC,3.54,212659.0,2293.0,26497.0,-0.40,1.09,12.46,13.68,37.0,0.09,1.52,17.79,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.79,43.0,H-LC,3.76,271440.0,6014.0,50786.0,-0.55,2.27,18.71,21.40,11.0,0.12,1.94,9.52,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,34.0,H-LC,12.46,274856.0,-71100.0,145949.0,0.34,-20.55,53.10,21.64,1.0,-0.49,1.97,1.23,X40,ATH,IT
83,VBL,671.64,-21.55,32.0,H-LC,9.54,282602.0,-33240.0,145907.0,-0.00,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,37.0,H-LC,10.28,219250.0,-22674.0,85946.0,0.50,-9.37,39.20,26.15,8.0,-0.26,1.57,3.61,X40,ATH,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
40,INFY,2275.00,-21.39,38.0,H-LC,8.25,306507.0,-6381.0,178081.0,-0.47,-2.04,58.10,54.87,3.0,-0.04,2.19,4.47,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,44.0,H-LC,1.74,156075.0,-30020.0,77507.0,0.00,-16.13,49.66,25.51,15.0,-0.39,1.12,15.02,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.29,30.0,H-LC,5.80,206114.0,-45654.0,98873.0,0.27,-18.13,47.97,21.13,27.0,-0.46,1.47,10.87,X40,ATH,PAINTS
19,CIPLA,1795.00,-21.84,45.0,H-LC,3.91,208514.0,4014.0,37407.0,0.59,1.96,17.94,20.25,10.0,0.11,1.49,10.87,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-15.17,39.0,H-LC,3.54,212659.0,2293.0,26497.0,-0.40,1.09,12.46,13.68,37.0,0.09,1.52,17.79,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,34.0,H-LC,12.46,274856.0,-71100.0,145949.0,0.34,-20.55,53.10,21.64,1.0,-0.49,1.97,1.23,X40,ATH,IT
40,INFY,2275.00,-21.39,38.0,H-LC,8.25,306507.0,-6381.0,178081.0,-0.47,-2.04,58.10,54.87,3.0,-0.04,2.19,4.47,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
83,VBL,671.64,-21.55,32.0,H-LC,9.54,282602.0,-33240.0,145907.0,-0.00,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,45.0,H-LC,7.66,249120.0,-12499.0,131710.0,-0.17,-4.78,52.87,45.57,7.0,-0.09,1.78,6.00,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,37.47,79526.0,-34023.0,74071.0,-0.00,-29.96,93.14,35.27,268.0,-0.46,0.57,94.64,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,47.0,H-SC,11.66,92550.0,-5430.0,27034.0,0.54,-5.54,29.21,22.05,152.0,-0.20,0.66,34.53,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.82,113685.0,13181.0,15927.0,0.00,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,65.0,H-SC,12.10,130221.0,-12258.0,31774.0,0.00,-8.60,24.40,13.70,163.0,-0.39,0.93,53.70,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-187.50,79.0,M-SC,9.60,151200.0,1980.0,142204.0,-0.93,1.33,94.05,96.62,208.0,0.01,1.08,63.63,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1254.07,70.0,M-SC,18.31,184095.0,2952.0,42508.0,0.57,1.63,23.09,25.10,235.0,0.07,1.32,47.53,XY24,NTT,HEALTHCARE
69,SURYODAY,240.0,73.25,81.0,H-SC,3.88,159110.0,-19961.0,85697.0,0.26,-11.15,53.86,36.71,135.0,-0.23,1.14,57.87,XR,NTT,BANKS
61,SAMMAANCAP,326.0,-187.50,79.0,M-SC,9.60,151200.0,1980.0,142204.0,-0.93,1.33,94.05,96.62,208.0,0.01,1.08,63.63,XY25,NTT,FINANCE
7,ATULAUTO,844.0,3629.73,52.0,M-SC,7.87,167552.0,-29580.0,119414.0,0.53,-15.01,71.27,45.57,236.0,-0.25,1.20,18.79,XY24,NTT,AUTO
78,UJJIVANSFB,60.0,106.53,65.0,H-SC,12.10,130221.0,-12258.0,31774.0,0.00,-8.60,24.40,13.70,163.0,-0.39,0.93,53.70,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.01
1,25,43.09
2,50,74.85


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.27
LC    32.06
MC    23.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.46
X40N     11.25
XR       11.10
XY25     10.42
AR        8.10
OX40N     7.78
X200      1.65
MH        1.62
X5K       1.50
SR        1.17
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.05
H-LC    25.46
H-MC    20.41
M-SC    13.79
M-LC     5.57
M-MC     2.97
L-SC     1.43
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.81,-6.30,41.39
IT,12.49,-20.49,86.18
FINANCE,10.42,-15.75,68.33
MISC,6.96,-18.08,83.72
BANKS,6.44,-10.84,69.15
PAINTS,5.55,-18.89,36.62
ELECTRICAL,5.33,-10.48,49.24
INSURANCE,3.75,-5.28,44.64
AUTO,3.59,-14.79,69.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3246770.0,22
XR,1358345.0,14
AR,1200459.0,9
X40,827195.0,10
X40N,631502.0,9
OX40N,558108.0,10
XY25,479395.0,7
SR,258536.0,2
X5K,107532.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3731036.0,29
M-SC,1504216.0,17
H-LC,1260952.0,15
H-MC,1251961.0,15
M-LC,371854.0,4
M-MC,367898.0,2
L-SC,260820.0,3
L-MC,60239.0,1
L-LC,40507.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230300.0      6
M-SC       XY24       847053.0      7
H-SC       AR         821682.0      5
           XR         782957.0      7
H-MC       XY24       573411.0      4
H-LC       X40        531876.0      5
M-MC       XY24       367898.0      2
H-SC       X40N       310623.0      4
           SR         258536.0      2
           OX40N      256043.0      4
H-LC       X40N       234400.0      3
H-MC       X40        219976.0      4
H-LC       AR         209217.0      2
H-MC       XY25       180825.0      2
L-SC       XR         172523.0      2
M-SC       AR         169560.0      2
M-LC       XY24       153327.0      2
M-SC       XY25       142204.0      1
           OX40N      138940.0      4
           XR         131116.0      2
H-LC       XY25       115859.0      3
M-LC       XR         110995.0      1
           X5K        107532.0      1
H-MC       XR         100515.0      1
H-LC       X200        94819.0      1
L-SC       OX40N       88297.0      1
H-MC       X40N        86479.0      2
M-SC       X40         75343.0      1
H-MC       OX40N       74828.0      1
H-LC       XY24        74781.0      1
H-SC       MH          70895.0      1
L-MC       XR          60239.0      1
L-LC       XY25        40507.0      1
H-MC       MH          15927.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 57.0 seconds
